In [59]:
from ultralytics import YOLO
import cv2
import numpy as np
import random, glob, os, time
 


file_path = r'C:\Users\prude\SynologyDrive\vscode\dataset_img\bike\moto_v1\*.jpg'
# r'C:\Users\prude\SynologyDrive\vscode\dataset_img\bike\moto_v1' 경로의 파일을 모두 불러오기
img_files = glob.glob(file_path)
file_path = random.choice(img_files)
# file_path = r'C:\Users\prude\SynologyDrive\vscode\dataset_img\bike\moto_v1\20240122_154218.jpg'
img = cv2.imread(file_path)

frame = img.copy()
# 가로 값이 500 이상이면 가로 값을 500으로 줄이고 세로 값도 비율에 맞게 줄인다.
if frame.shape[1] > 500:
    scale = 500 / frame.shape[1]
    frame = cv2.resize(frame, dsize=(0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
# 윈도우 생성 및 이름 지정
str = 'image'
cv2.namedWindow(str)
cv2.moveWindow(str, 100, 100)
cv2.imshow(str, frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
model = YOLO(r'C:\Users\prude\SynologyDrive\vscode\my_project\qt_django_yolo\rsc\models\motobike_e300_b8_s640.pt')
print(frame.shape)
# 임계값 설정
THR = 0.7
# 디텍션
detection = model(img)[0] # 사진은 한 장이라서 0번 슬라이싱 밖에 없음
labels = detection.names
# yolo result 객체의 boxes 속성에는 xmin, ymin, xmax, ymax, confidence_score, class_id 값이 담겨 있음
detect_num = 0
for data in detection.boxes.data.tolist(): # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    # 임계값 이하는 생략 하라는 코드
    confidence = float(data[4]) 
    if confidence < THR:
        continue
    # 오토바이만 탐지하도록 설정
    label = int(data[5])
    if label != 0:
        continue
    xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    # xmin, ymin, xmax, ymax 이미지만 남기기
    bike_img = img[ymin:ymax, xmin:xmax]
    detect_num += 1
cv2.imshow('bike', bike_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


(504, 500, 3)

0: 640x640 1 Bike, 1 Bike_Number_Plate, 138.0ms
Speed: 4.0ms preprocess, 138.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [57]:
# 번호판 탐지
print('번호판 탐지')
THR = 0.3
# 디텍션
detection = model(bike_img)[0] # 사진은 한 장이라서 0번 슬라이싱 밖에 없음
for data in detection.boxes.data.tolist(): # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    # 임계값 이하는 생략 하라는 코드
    confidence = float(data[4]) 
    if confidence < THR:
        continue
    # 번호판만 탐지하도록 설정
    label = int(data[5])
    if label != 1:
        continue
    xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    nbp_img = bike_img[ymin:ymax, xmin:xmax]

cv2.imshow('nbp', nbp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

번호판 탐지

0: 640x320 2 Bikes, 1 Bike_Number_Plate, 72.3ms
Speed: 1.0ms preprocess, 72.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 320)


In [58]:
# 번호판 처리
print('번호판 처리')

def reorderPts(pts):
    idx = np.lexsort((pts[:, 1], pts[:, 0]))  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
    pts = pts[idx]  # x좌표로 정렬

    if pts[0, 1] > pts[1, 1]:
        pts[[0, 1]] = pts[[1, 0]]

    if pts[2, 1] < pts[3, 1]:
        pts[[2, 3]] = pts[[3, 2]]

    return pts

def nbp_transform(frame):
    # 출력 영상 설정
    dw, dh = 300, 150
    srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32)
    dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32)
    dst = np.zeros((dh, dw), np.uint8)

    frame = cv2.GaussianBlur(frame, (3,3), 5)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # frame = cv2.Canny(frame, 50, 200)
    th, frame = cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 외곽선 검출 및 명함 검출
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for pts in contours:
        # 너무 작은 객체는 제외
        if cv2.contourArea(pts) < 10:
            continue
        # 외곽선 근사화
        approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
        # 컨벡스가 아니면 제외
        if not cv2.isContourConvex(approx) or len(approx) != 4:
            continue
        # cv2.polylines(frame, [approx], True, (0, 255, 0), 2, cv2.LINE_AA)
        srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))

        pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
        dst = cv2.warpPerspective(frame, pers, (dw, dh), flags=cv2.INTER_CUBIC)
    return dst

cv2.imshow('nbp', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 3. 윤곽선 추출
# contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # 4. 사각형 근사
# areas = []
# for contour in contours:
#     approx = cv2.approxPolyDP(contour, 0.1 * cv2.arcLength(contour, True), True)
#     # 5. 필터링
#     if len(approx) == 4:
#         color = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
#         x, y, w, h = cv2.boundingRect(approx)
#         rect_points = approx.reshape(4, 2)
#         # cv2.polylines(img, [rect_points], True, color, 2)
#         sum_angle = calc_angle(rect_points)
#         # x, y, w, h 값을 이용해서 사각형중 넓이가 가장 큰 것만 찾기
#         # 사각형 넓이 구하기
#         area = w * h
#         areas.append((area, rect_points, sum_angle, x, y, w, h))
# # areas 각 원소중 sum_angle 값이 360과 가장 가까운 값을 찾기
# areas = sorted(areas, key=lambda x: abs(x[2] - 360))
# try:
#     x, y, w, h = areas[0][3:]
#     # 사각형의 꼭지점 좌표 찾기
#     rect_points = areas[0][1]
#     # rect_points를 이용해서 다각형 그리기
#     cv2.polylines(img, [rect_points], True, (0, 255, 0), 2)
# except:
#     print('사각형 없음')


번호판 처리


In [4]:
# 번호판 처리
# 좌표값 4개를 받아서 각도를 계산하는 함수

def calc_angle(points):
    # 각도 계산
    angle = 0
    for i in range(4):
        p1 = points[i]
        p2 = points[(i+1) % 4]
        p3 = points[(i+2) % 4]
        v1 = p1 - p2
        v2 = p3 - p2
        # 벡터의 내적
        dot = np.dot(v1, v2)
        # 벡터의 크기
        norm1 = np.linalg.norm(v1)
        norm2 = np.linalg.norm(v2)
        # 각도 계산
        angle += np.arccos(dot / (norm1 * norm2)) * 180 / np.pi
    return angle


In [5]:

# 확인된 컨투어를 기준으로 사각형 탐지
def find_rect(img):
    frame = img.copy()
    frame = cv2.GaussianBlur(frame, (3,3), 5)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # frame = cv2.Canny(frame, 50, 200)
    # plot(frame, 'canny')
    # 3. 윤곽선 추출
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 4. 사각형 근사
    areas = []
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.1 * cv2.arcLength(contour, True), True)
        # 5. 필터링
        if len(approx) == 4:
            color = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
            x, y, w, h = cv2.boundingRect(approx)
            rect_points = approx.reshape(4, 2)
            # cv2.polylines(img, [rect_points], True, color, 2)
            sum_angle = calc_angle(rect_points)
            # x, y, w, h 값을 이용해서 사각형중 넓이가 가장 큰 것만 찾기
            # 사각형 넓이 구하기
            area = w * h
            areas.append((area, rect_points, sum_angle, x, y, w, h))
    # areas 각 원소중 sum_angle 값이 360과 가장 가까운 값을 찾기
    areas = sorted(areas, key=lambda x: abs(x[2] - 360))
    try:
        x, y, w, h = areas[0][3:]
        # 사각형의 꼭지점 좌표 찾기
        rect_points = areas[0][1]
        # rect_points를 이용해서 다각형 그리기
        cv2.polylines(img, [rect_points], True, (0, 255, 0), 2)
    except:
        print('사각형 없음')
    return img, rect_points


In [6]:
# 변형 (아핀, 원근)
def affine_transform(img, points):
    # 변환 전 4개 좌표
    pts1 = np.float32(points)
    # 변환 후 4개 좌표
    pts2 = np.float32([[0, 0], [350, 0], [350, 100], [0, 100]])
    # 변환 행렬
    M = cv2.getPerspectiveTransform(pts1, pts2)
    # 원근 변환
    result = cv2.warpPerspective(img, M, (350, 100))
    return result
    
## 이미지가 반전되는 문제가 있음 수정요망

# 이미지 바이너리화
def binary_transform(img, threshold):
    # 이미지 이진화
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

In [7]:
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
from paddleocr import PaddleOCR   
 
def paddle_ocr(img):
    Ocr = PaddleOCR(use_angle_cls=True, lang='korean') # need to run only once to download and load model into memory
    result = Ocr.ocr(img, cls=True)
    result = result[0]
    boxes = [line[0] for line in result]
    texts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    return img, boxes, texts, scores

def plot_nbp(img, boxes, texts, scores):
    for box, txt, score in zip(boxes, texts, scores):
        # cv2.putText(image, txt, (box[0][0], box[1][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.polylines(img, np.array([box], dtype=np.int32), True, (0, 255, 0), 1)
    plot(img, 'nbp_plot')

In [8]:
# 적용해보기
def main(img):
    plot(img, 'original')
    frame, labels, detect_num = detect_bike(img)
    if detect_num == 0:
        print('fail_detect_bike')
        plot(img, 'fail_detect_bike')
        return 
    plot(frame, 'success_detect_bike')
    # 번호판 탐지
    frame, bool = detect_license_plate(frame)
    if bool == False:
        print('fail_detect_license_plate')
        return
    plot(frame, 'detect_license_plate')
    # 번호판 이진화
    # frame = binary_transform(frame, 3) # [0.7315500974655151, 0.9999265670776367]
    # plot(frame, 'binary_transform')

    print('PaddleOCR')
    img, boxes, texts, scores = paddle_ocr(frame)
    print(boxes, texts, scores)
    plot_nbp(frame, boxes, texts, scores)
    return frame

C:\Users\prude\SynologyDrive\vscode\dataset_img\bike\moto_v1\20240122_154218.jpg

0: 640x480 2 Bikes, 1 Bike_Number_Plate, 121.0ms
Speed: 4.0ms preprocess, 121.0ms inference, 1031.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x512 1 Bike, 1 Bike_Number_Plate, 114.0ms
Speed: 2.0ms preprocess, 114.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)
PaddleOCR
[2024/07/04 22:07:45] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\prude/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\U

: 